In [1]:
import plotly.graph_objects as go 
from plotly.subplots import make_subplots
import aerosandbox.numpy as anp
import aerosandbox as asb 
from surrogate import Surrogate2Callback,Surrogate,KRG

In [2]:
xt = anp.array([0.0, 1.0, 2.0, 3.0, 4.0])
yt = anp.array([0.0, 1.0, 1.5, 0.9, 1.0])

sm = KRG(theta0=[1e-2],print_global=False)
sm.set_training_values(xt, yt)
sm.train()

sm = Surrogate(sm)
sm_predict = Surrogate2Callback("sm", surrogate=sm)

num = 100
x = anp.linspace(0.0, 4.0, num)
y=sm_predict(x).toarray()

fig=make_subplots(1,1)
fig.add_scatter(x=x,y=y[:,0],mode="lines+markers",row=1,col=1,line={"shape":"spline","width":2})
fig.update_layout(width=600,margin={"r":20,"l":20,"t":20,"b":20})
fig.show(config={"scrollZoom":True})

In [3]:
opti=asb.Opti()
x=opti.variable(init_guess=1.0,lower_bound=0.0,upper_bound=4.0)
obj=sm_predict(x)
opti.maximize(obj)
sol=opti.solve()
print(sol(x),sol(obj))

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.4.1.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        2
Number of nonzeros in Lagrangian Hessian.............:        1



IndexError: index 1 is out of bounds for axis 1 with size 1

Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!


Function jac_sm (0x17fc79bf640)
Input 0 (i0): 1
Input 1 (i1): 1
Function nlp_grad_f (0x17fc79c2080)
Input 0 (x): 1
Input 1 (p): NULL
CasADi - 2025-06-27 14:12:21 WARNING("IpoptUserClass::eval_grad_f failed:.../casadi/core/oracle_function.cpp:367: Error in solver:nlp_grad_f:Error in Function::operator() for 'nlp_grad_f' [MXFunction] at .../casadi/core/function.cpp:1539:
Error in Function::operator() for 'jac_sm' [CallbackInternal] at .../casadi/core/function.cpp:1539:
.../casadi/core/function_internal.cpp:3723: Failed to evaluate 'eval_dm' for jac_sm:
.../casadi/core/callback_internal.cpp:116: Error calling "eval" for object jac_sm:
/work/swig/python/target3/source/casadiPYTHON_wrap_gil_release.cxx:3855: index 1 is out of bounds for axis 1 with size 1") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:118]


IndexError: index 1 is out of bounds for axis 1 with size 1


Number of Iterations....: 0

Number of objective function evaluations             = 0
Number of objective gradient evaluations             = 1
Number of equality constraint evaluations            = 0
Number of inequality constraint evaluations          = 1
Number of equality constraint Jacobian evaluations   = 0
Number of inequality constraint Jacobian evaluations = 1
Number of Lagrangian Hessian evaluations             = 0
Total seconds in IPOPT                               = 0.422

EXIT: Invalid number in NLP function or derivative detected.
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_g  |        0 (       0)   4.00us (  4.00us)         1
  nlp_grad_f  | 419.00ms (209.50ms) 419.33ms (209.67ms)         2
   nlp_jac_g  |        0 (       0)  28.00us ( 14.00us)         2
       total  | 424.00ms (424.00ms) 423.47ms (423.47ms)         1


Function jac_sm (0x17fc79bf640)
Input 0 (i0): 1
Input 1 (i1): 1
Function nlp_grad_f (0x17fc79c2080)
Input 0 (x): 1
Input 1 (p): NULL
CasADi - 2025-06-27 14:12:21 WARNING("IpoptUserClass::eval_grad_f failed:.../casadi/core/oracle_function.cpp:367: Error in solver:nlp_grad_f:Error in Function::operator() for 'nlp_grad_f' [MXFunction] at .../casadi/core/function.cpp:1539:
Error in Function::operator() for 'jac_sm' [CallbackInternal] at .../casadi/core/function.cpp:1539:
.../casadi/core/function_internal.cpp:3723: Failed to evaluate 'eval_dm' for jac_sm:
.../casadi/core/callback_internal.cpp:116: Error calling "eval" for object jac_sm:
/work/swig/python/target3/source/casadiPYTHON_wrap_gil_release.cxx:3855: index 1 is out of bounds for axis 1 with size 1") [.../casadi/interfaces/ipopt/ipopt_nlp.cpp:118]


RuntimeError: Error in Opti::solve [OptiNode] at .../casadi/core/optistack.cpp:217:
.../casadi/core/optistack_internal.cpp:1334: Assertion "return_success(accept_limit)" failed:
Solver failed. You may use opti.debug.value to investigate the latest values of variables. return_status is 'Invalid_Number_Detected'